In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import requests
import json
from openai import OpenAI
import configparser
import argparse

import tools.ProcessFiles as pf

In [3]:
excel = pf.get_excel_data(1)
json_files = pf.get_filelist(1, pf.FINAL_DATA_PATH)

In [5]:
excel[excel['분류'] != 'Lv5']

,ISBN,도서명,출판일,코퍼스 1분류,코퍼스 2분류,비고,분류
관리번호,,,,,,,
225933255,9791163478782,금융의 현재와 미래,2020-01-03,금융,기초,시사,Lv3/4
239373741,9788984059900,요즘 금리 쉬운 경제,2020-04-27,금융,기초,시사,Lv3/4
243228629,9788960542570,알기 쉬운 보험 세테크 100% 활용법,2020-06-23,보험,기초,세무,Lv3/4
248779244,9788957614358,공정거래 용어사전,2020-08-10,금융,기초,용어,Lv2
250217022,9791190820660,1페이지 보험 정리의 기술,2020-08-28,보험,기초,시사,Lv3/4
...,...,...,...,...,...,...,...
358558337,9791194682431,현명한 대출 관리,2025-02-26,카드,기초,시사,Lv2
358558342,9791194682479,신용 관리의 모든 것,2025-02-26,카드,기초,시사,Lv2
358558344,9791194682486,부채 탈출의 기술,2025-02-26,카드,기초,시사,Lv2


In [ ]:
os.path.splitext(os.path.basename(json_files[0]))

In [ ]:
sorted(lv234_qna)

In [68]:
multiple, short, essay = [],[],[]
origin = json.load(open(data_dir+'/merged_qna_set.json', 'r', encoding='utf-8'))
for qna in origin:
    if qna.get('file_id') in lv234_qna:
        if qna.get('qna_type') == "multiple-choice":
            multiple.append(qna)
        elif qna.get('qna_type') == "short-answer":
            short.append(qna)
        elif qna.get('qna_type') == "essay":
            essay.append(qna)
# data_dir = os.path.join(base_dir,'Lv2', 're')
# origin = json.load(open(os.path.join(base_dir, 'merged_qna_set.json'), 'r', encoding='utf-8'))

In [ ]:
short

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--model_name", type=str)
# parser.add_argument("--task_files", type=str)
# parser.add_argument("--config_path", type=str)
# parser.add_argument("--res_path", type=str)
# parser.add_argument("--api_type", type=str)
# args = parser.parse_args()

In [ ]:
models = ['anthropic/claude-sonnet-4', 'google/gemini-2.5-flash', 'x-ai/grok-4-fast:free', 'openai/gpt-4.1', 'openai/gpt-oss-120b:free', 'openai/gpt-5', 'qwen/qwen3-next-80b-a3b-thinking', 'qwen/qwen3-235b-a22b-thinking-2507']

In [13]:
question = """아래 질문을 읽고, 
어떤 재화 시장에서 소비자잉여와 생산자잉여에 대한 설명으로 옳은 것은?(단, 수요곡선은 우하향하며, 공급곡선은 우상향한다.)\n\n① 소비자잉여는 실제로 지불한 금액이 지불할 용의가 있는 최대 금액을 초과하는 부분이다.\n② 소비자잉여는 소비자가 재화의 소비에서 얻는 편익의 총합과 같다.\n③ 고정비용이 없는 장기에 생산자잉여는 기업의 이윤과 같다.\n④ 기업에 단위당 T원의 물품세를 부과하면 가격이 상승하여 생산자잉여가 증가한다.
"""

In [ ]:
llm_result

In [ ]:
# def query_model_openrouter(model_name, config, task_file, retry = 3):
#     client = OpenAI(api_key=config["OPENROUTER"]["key"], base_url=config["OPENROUTER"]["url"])
#     df_task = pd.read_csv(task_file, encoding='utf-8')

#     task_name = task_file.split('/')[-1].split('.')[0]
#     questions = list(df_task["instruction"] + "\n\n" + df_task["question"])

#     results = []
#     for question in tqdm(questions, desc=task_name):
#         success = False
#         trial = 0
#         while(not success and trial < retry):
#             try:
#                 response = client.chat.completions.create(
#                     model=model_name,
#                     temperature=float(config["PARAMS"]["temperature"]),
#                     frequency_penalty=float(config["PARAMS"]["frequency_penalty"]), 
#                     presence_penalty=float(config["PARAMS"]["presence_penalty"]),
#                     top_p=float(config["PARAMS"]["top_p"]), 
#                     max_tokens=int(config["PARAMS"]["max_tokens"]), 
#                     messages=[{"role": "user", "content": question}],
#                 )
#                 llm_result = response.choices[0].message.content

#                 # response = requests.post(
#                 #     url=config["OPENROUTER"]["url"] + "/chat/completions",
#                 #     headers={
#                 #         "Authorization": "Bearer " + config["OPENROUTER"]["key"],
#                 #     },
#                 #     data=json.dumps({
#                 #         "model": model_name, 
#                 #         "messages": [
#                 #         {
#                 #             "role": "user",
#                 #             "content": question
#                 #         }
#                 #         ],
#                 #         "provider":{
#                 #             "order": ["Together", ], 
#                 #         },
#                 #         "temperature":float(config["PARAMS"]["temperature"]),
#                 #         "frequency_penalty":float(config["PARAMS"]["frequency_penalty"]),
#                 #         "presence_penalty":float(config["PARAMS"]["presence_penalty"]),
#                 #         "top_p":float(config["PARAMS"]["top_p"]),
#                 #         "max_tokens":int(config["PARAMS"]["max_tokens"]),
#                 #     }),
#                 #     timeout=180
#                 # )
#                 # response = response.json()
#                 # llm_result = response["choices"][0]["message"]["content"]

#                 results.append((question, llm_result))
#                 success = True
#                 trial += 1
#             except Exception as e:
#                 success = False
#                 trial += 1
#                 print(f"Trial:{trial}\nError log:\n{e}")
#                 continue
#         if not success:
#             results.append((question, "API ERROR"))

#     df_results = pd.DataFrame(results, columns=["prompt", "response"])
#     df_merge = pd.concat([df_task, df_results], axis=1)

#     return df_merge